In [7]:
import json
import requests

import pandas as pd

from datetime import datetime
from random import randint
from time import sleep


In [ ]:
data = []

for page in range(1, 10):
    ts = datetime.now().isoformat()
    payload = json.dumps(
        requests.get(url="https://catfact.ninja/facts", params={"page": page}).json()
    )

    data.append({"ts": ts, "payload": payload})

    sleep(randint(1, 5))

In [14]:
pdf = pd.DataFrame(data=data)
pdf


,ts,payload
0,2023-07-13T17:26:59.345122,"{""current_page"": 1, ""data"": [{""fact"": ""Unlike ..."
1,2023-07-13T17:27:01.746471,"{""current_page"": 2, ""data"": [{""fact"": ""During ..."
2,2023-07-13T17:27:05.066221,"{""current_page"": 3, ""data"": [{""fact"": ""A cat c..."
3,2023-07-13T17:27:07.392719,"{""current_page"": 4, ""data"": [{""fact"": ""A cat\u..."
4,2023-07-13T17:27:12.699642,"{""current_page"": 5, ""data"": [{""fact"": ""A cat c..."
5,2023-07-13T17:27:16.992408,"{""current_page"": 6, ""data"": [{""fact"": ""In the ..."
6,2023-07-13T17:27:19.372787,"{""current_page"": 7, ""data"": [{""fact"": ""All cat..."
7,2023-07-13T17:27:21.721222,"{""current_page"": 8, ""data"": [{""fact"": ""Approxi..."
8,2023-07-13T17:27:26.052877,"{""current_page"": 9, ""data"": [{""fact"": ""A cat\u..."


In [15]:
pdf.to_csv("./dataset.csv", index_label="id")


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("escape", '"')
    .load("./dataset.csv")
)

print(df.schema)
df.show()

23/07/13 23:39:08 WARN Utils: Your hostname, laptop resolves to a loopback address: 127.0.1.1; using 192.168.2.5 instead (on interface wlan0)
23/07/13 23:39:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 23:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/13 23:39:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


StructType([StructField('id', StringType(), True), StructField('ts', StringType(), True), StructField('payload', StringType(), True)])
+---+--------------------+--------------------+
| id|                  ts|             payload|
+---+--------------------+--------------------+
|  0|2023-07-13T17:26:...|{"current_page": ...|
|  1|2023-07-13T17:27:...|{"current_page": ...|
|  2|2023-07-13T17:27:...|{"current_page": ...|
|  3|2023-07-13T17:27:...|{"current_page": ...|
|  4|2023-07-13T17:27:...|{"current_page": ...|
|  5|2023-07-13T17:27:...|{"current_page": ...|
|  6|2023-07-13T17:27:...|{"current_page": ...|
|  7|2023-07-13T17:27:...|{"current_page": ...|
|  8|2023-07-13T17:27:...|{"current_page": ...|
+---+--------------------+--------------------+



In [2]:
from pushcart.metadata import Metadata

md = Metadata(df, infer_fraction=1.0)
md.get_metadata()


INFO:pushcart.metadata:Attempting to infer timestamp format for id column.
INFO:pushcart.metadata:Attempting to infer JSON schema for id column.
INFO:pushcart.metadata:Attempting to infer timestamp format for ts column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload column.


DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all'}, corner_renderer=None, default_renderer=TextRe…

In [3]:
# Edits are reflected in the underlying dataset
md.metadata_df


,column_order,source_column_name,source_column_type,dest_column_name,dest_column_type,transform_function,default_value,validation_rule,validation_action
key,,,,,,,,,
0,0,id,string,id,int,,,,
1,1,ts,string,ts,string,"F.to_timestamp(F.col(""ts""), ""yyyy-MM-dd'T'HH:m...",,,
2,2,payload,string,payload,string,"F.from_json(F.col(""payload""), schema=""struct<c...",,,


In [4]:
code = md.generate_code()


INFO:pushcart.metadata:Excluding technical columns: None
INFO:pushcart.metadata:
df = (df
	.withColumn("id", F.col("id").cast("int"))
	.withColumn("ts", F.to_timestamp(F.col("ts"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
	.withColumn("payload", F.from_json(F.col("payload"), schema="struct<current_page:bigint,data:array<struct<fact:string,length:bigint>>,first_page_url:string,from:bigint,last_page:bigint,last_page_url:string,links:array<struct<active:boolean,label:string,url:string>>,next_page_url:string,path:string,per_page:bigint,prev_page_url:string,to:bigint,total:bigint>"))
	.select(['id', 'ts', 'payload']))


23/07/13 23:39:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
import pyspark.sql.functions as F

processed_df = (
    df.withColumn("id", F.col("id").cast("int"))
    .withColumn("ts", F.to_timestamp(F.col("ts"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
    .withColumn(
        "payload",
        F.from_json(
            F.col("payload"),
            schema="struct<current_page:bigint,data:array<struct<fact:string,length:bigint>>,first_page_url:string,from:bigint,last_page:bigint,last_page_url:string,links:array<struct<active:boolean,label:string,url:string>>,next_page_url:string,path:string,per_page:bigint,prev_page_url:string,to:bigint,total:bigint>",
        ),
    )
    .select(["id", "ts", "payload"])
)

print(processed_df.schema)
processed_df.show()

StructType([StructField('id', IntegerType(), True), StructField('ts', TimestampType(), True), StructField('payload', StructType([StructField('current_page', LongType(), True), StructField('data', ArrayType(StructType([StructField('fact', StringType(), True), StructField('length', LongType(), True)]), True), True), StructField('first_page_url', StringType(), True), StructField('from', LongType(), True), StructField('last_page', LongType(), True), StructField('last_page_url', StringType(), True), StructField('links', ArrayType(StructType([StructField('active', BooleanType(), True), StructField('label', StringType(), True), StructField('url', StringType(), True)]), True), True), StructField('next_page_url', StringType(), True), StructField('path', StringType(), True), StructField('per_page', LongType(), True), StructField('prev_page_url', StringType(), True), StructField('to', LongType(), True), StructField('total', LongType(), True)]), True)])
+---+--------------------+------------------

In [6]:
nested_md = Metadata(processed_df, infer_fraction=1.0)
nested_md.get_metadata()


INFO:pushcart.metadata:Attempting to infer timestamp format for payload.data.fact column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload.data.fact column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload.first_page_url column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload.first_page_url column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload.last_page_url column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload.last_page_url column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload.links.label column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload.links.label column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload.links.url column.
INFO:pushcart.metadata:Attempting to infer JSON schema for payload.links.url column.
INFO:pushcart.metadata:Attempting to infer timestamp format for payload.next_page_url column.
INFO:push

DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all'}, corner_renderer=None, default_renderer=TextRe…

In [7]:
flat_code = nested_md.generate_code()


INFO:pushcart.metadata:Excluding technical columns: None
INFO:pushcart.metadata:
df = (df
	.withColumn("payload_current_page", F.col("payload.current_page").cast("bigint"))
	.withColumn("payload_data", F.explode("payload.data"))
	.withColumn("payload_data_fact", F.col("payload_data.fact").cast("string"))
	.withColumn("payload_data_length", F.col("payload_data.length").cast("bigint"))
	.withColumn("payload_first_page_url", F.col("payload.first_page_url").cast("string"))
	.withColumn("payload_from", F.col("payload.from").cast("bigint"))
	.withColumn("payload_last_page", F.col("payload.last_page").cast("bigint"))
	.withColumn("payload_last_page_url", F.col("payload.last_page_url").cast("string"))
	.withColumn("payload_links", F.explode("payload.links"))
	.withColumn("payload_links_active", F.col("payload_links.active").cast("boolean"))
	.withColumn("payload_links_label", F.col("payload_links.label").cast("string"))
	.withColumn("payload_links_url", F.col("payload_links.url").cast("string"

In [8]:
flattened_df = (
    processed_df.withColumn(
        "payload_current_page", F.col("payload.current_page").cast("bigint")
    )
    .withColumn("payload_data", F.explode("payload.data"))
    .withColumn("payload_data_fact", F.col("payload_data.fact").cast("string"))
    .withColumn("payload_data_length", F.col("payload_data.length").cast("bigint"))
    .withColumn(
        "payload_first_page_url", F.col("payload.first_page_url").cast("string")
    )
    .withColumn("payload_from", F.col("payload.from").cast("bigint"))
    .withColumn("payload_last_page", F.col("payload.last_page").cast("bigint"))
    .withColumn("payload_last_page_url", F.col("payload.last_page_url").cast("string"))
    .withColumn("payload_links", F.explode("payload.links"))
    .withColumn("payload_links_active", F.col("payload_links.active").cast("boolean"))
    .withColumn("payload_links_label", F.col("payload_links.label").cast("string"))
    .withColumn("payload_links_url", F.col("payload_links.url").cast("string"))
    .withColumn("payload_next_page_url", F.col("payload.next_page_url").cast("string"))
    .withColumn("payload_path", F.col("payload.path").cast("string"))
    .withColumn("payload_per_page", F.col("payload.per_page").cast("bigint"))
    .withColumn("payload_prev_page_url", F.col("payload.prev_page_url").cast("string"))
    .withColumn("payload_to", F.col("payload.to").cast("bigint"))
    .withColumn("payload_total", F.col("payload.total").cast("bigint"))
    .select(
        [
            "id",
            "ts",
            "payload_current_page",
            "payload_data",
            "payload_data_fact",
            "payload_data_length",
            "payload_first_page_url",
            "payload_from",
            "payload_last_page",
            "payload_last_page_url",
            "payload_links",
            "payload_links_active",
            "payload_links_label",
            "payload_links_url",
            "payload_next_page_url",
            "payload_path",
            "payload_per_page",
            "payload_prev_page_url",
            "payload_to",
            "payload_total",
        ]
    )
)

flattened_df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------+-----------------+---------------------+--------------------+--------------------+-------------------+--------------------+---------------------+--------------------+----------------+---------------------+----------+-------------+
| id|                  ts|payload_current_page|        payload_data|   payload_data_fact|payload_data_length|payload_first_page_url|payload_from|payload_last_page|payload_last_page_url|       payload_links|payload_links_active|payload_links_label|   payload_links_url|payload_next_page_url|        payload_path|payload_per_page|payload_prev_page_url|payload_to|payload_total|
+---+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------+-----------------+---------------------+--------------------+--------------------+-------------------+-